In [1]:
import numpy
from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Embedding
from keras.preprocessing import sequence
numpy.random.seed(7)

ImportError: cannot import name 'runtime_version' from 'google.protobuf' (C:\Users\gleb\AppData\Local\Programs\Python\Python39\lib\site-packages\google\protobuf\__init__.py)

In [3]:
top_words = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=top_words)

NameError: name 'imdb' is not defined

In [ ]:
max_review_length = 500
X_train = sequence.pad_sequences(X_train, maxlen=max_review_length)
X_test = sequence.pad_sequences(X_test, maxlen=max_review_length)

In [ ]:
embedding_vecor_length = 32
model = Sequential()
model.add(Embedding(top_words, embedding_vecor_length, input_length= max_review_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())
model.fit(X_train, y_train, epochs=3, batch_size=64)

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/embedding.py:97: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm (LSTM)                     │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None
Epoch 1/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 224s 567ms/step - accuracy: 0.6620 - loss: 0.5828
Epoch 2/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 223s 569ms/step - accuracy: 0.8769 - loss: 0.3062
Epoch 3/3
391/391 ━━━━━━━━━━━━━━━━━━━━ 265s 576ms/step - accuracy: 0.8852 - loss: 0.2820


In [ ]:
scores = model.evaluate(X_test, y_test, verbose=0)
print("Accuracy: %.2f%%" % (scores[1]*100))

Accuracy: 86.75%


In [ ]:
import numpy as np
from keras.preprocessing import sequence
from keras.datasets import imdb

word_to_index = imdb.get_word_index()
top_words = 5000

def preprocess_text(text):
    text = text.lower().replace('<br />', ' ')
    words = text.split()

    word_indices = []
    for word in words:
        index = word_to_index.get(word)
        if index is not None and index < top_words:
            word_indices.append(index + 3)

    return word_indices

def predict_sentiment(model, text, max_review_length=500):
    word_indices = preprocess_text(text)

    if len(word_indices) == 0:
        return "no data for recog", 0.5

    sequences = [word_indices]
    padded_sequences = sequence.pad_sequences(sequences, maxlen=max_review_length)

    prediction = model.predict(padded_sequences, verbose=0)[0][0]

    if prediction > 0.7:
        sentiment = "POS"
        confidence = prediction
    else:
        sentiment = "NEG"
        confidence = 0.5

    return sentiment, prediction, confidence

def test_single_comment(model):
    while True:
        comment = input("\nInput comment for analysis ('quit' for exit): ")

        if comment.lower() == 'quit':
            break

        sentiment, score, confidence = predict_sentiment(model, comment)

        print(f"\nResults:")
        print(f"Emotion recognized: {sentiment}")
        print(f"Model score: {score:.4f}")
        print(f"Confidence: {confidence:.2%}")

def test_multiple_comments(model):
    test_comments = [
        "This movie was absolutely fantastic! Great acting and storyline.",
        "I hated this film. It was boring and poorly made.",
        "The acting was good but the plot was weak.",
        "One of the best movies I've ever seen!",
        "Terrible waste of time. Don't watch it.",
        "It was okay, nothing special.",
        "Brilliant cinematography and excellent performances.",
        "Poor direction and bad screenplay ruined the movie.",
        "I loved the characters and the emotional depth.",
        "Boring and predictable from start to finish."
    ]


    for i, comment in enumerate(test_comments, 1):
        sentiment, score, confidence = predict_sentiment(model, comment)

        print(f"\n{i}. {comment}")
        print(f"Result: {sentiment} (оценка: {score:.4f})")
        print(f"Confidence: {confidence:.2%}")

def batch_predict_sentiment(model, comments_list):
    results = []

    for comment in comments_list:
        sentiment, score, confidence = predict_sentiment(model, comment)
        results.append({
            'comment': comment,
            'sentiment': sentiment,
            'score': score,
            'confidence': confidence
        })

        print(f"\nComment: {comment}")
        print(f"Result: {sentiment} | Score: {score:.4f} | Confidence: {confidence:.2%}")

    return results

def main():
    while True:
        print("1 - one comment")
        print("2 - set of comments")
        print("3 - comments from external file")
        print("4 - exit")

        choice = input("type (1-4): ").strip()

        if choice == '1':
            test_single_comment(model)
        elif choice == '2':
            test_multiple_comments(model)
        elif choice == '3':
            custom_comments = [
                "Amazing film with great performances!",
                "Not my cup of tea, too slow and boring.",
                "The director did a wonderful job with this movie."
            ]
            batch_predict_sentiment(model, custom_comments)
        elif choice == '4':
            print("exit")
            break
        else:
            print("1-4.")

def evaluate_model_performance(model, X_test, y_test):
    scores = model.evaluate(X_test, y_test, verbose=0)
    print(f"test acc: {scores[1]:.2%}")
    print(f"test loss: {scores[0]:.4f}")

def save_model(model, filename='sentiment_model.h5'):
    model.save(filename)

if __name__ == "__main__":
    main()

1 - one comment
2 - set of comments
3 - comments from external file
4 - exit
type (1-4): 1

Input comment for analysis ('quit' for exit): One of the best movies I've ever seen!

Results:
Emotion recognized: POS
Model score: 0.8935
Confidence: 89.35%

Input comment for analysis ('quit' for exit): Awful cast

Results:
Emotion recognized: NEG
Model score: 0.6435
Confidence: 50.00%

Input comment for analysis ('quit' for exit): quit
1 - one comment
2 - set of comments
3 - comments from external file
4 - exit
type (1-4): quit
1-4.
1 - one comment
2 - set of comments
3 - comments from external file
4 - exit


KeyboardInterrupt: Interrupted by user